In [57]:
import joblib
import sys
import os
alamat = "/".join(os.getcwd().split("\\")[:-2])
sys.path.append(alamat+'/praproses')
import ngrams as ng
from normalisasi import normalisasi as nrm
import token as tkn
import stemming as stm
import cleansing
import stemming
import seleksi_kata as sk
import pandas as pd
import isDigit as isd

In [58]:
tdidf = joblib.load("model/tdidf.w")
naive_bayes = joblib.load("model/naive_bayes.m")

In [59]:
def praproses(twt, n=3):
    twt = sk.seleksi(twt)
    twt = cleansing.cleaning(twt)
    twt = isd.remove_digit(twt)
    twt = nrm.slangword(twt)
    twt = stm.stemming(twt)
    twt = stm.StopWordRemover(twt)
    twt = ng.ngramku(twt, n=n)['string']
    return twt

In [60]:
praproses("RT @hpsetyawan: @afiadi setuju bgt", n=1)

'tuju banget'

In [61]:
kalimat = "RT @hpsetyawan: @afiadi setuju bgt"
# print(praproses(kalimat))
naive_bayes.predict(tdidf.transform([praproses(kalimat)]))[0]
#memprediksi kalimat di preproses, kemudian di transform menjadi vector,diprediksi menggunakan NB

'positive'

In [62]:
pwd

'G:\\Kuliah\\THESIS\\PORGRAM_MERDEKA\\Sentiment_merdeka_belajar\\N=3\\MNB'

# Prediksi data uji [Merdeka Belajar]

In [63]:
df = pd.read_excel("../../data_uji/PPDB ZONASI.xlsx")
twet_uji = df.Twit.tolist()
label_asli = df.label.tolist()
twet_uji_bersih = list()

for ix, t in enumerate(twet_uji):
    twet_uji_bersih.append(praproses(t))
#     print(ix)
prediksi = naive_bayes.predict(tdidf.transform(twet_uji_bersih))

from sklearn.metrics import classification_report
y_true = label_asli
y_pred = prediksi
target_names = ['negative', 'positive',]

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred,labels=["positive", "negative"]))
print(classification_report(y_true, y_pred, target_names=target_names))

[[46  4]
 [ 2 48]]
              precision    recall  f1-score   support

    negative       0.92      0.96      0.94        50
    positive       0.96      0.92      0.94        50

    accuracy                           0.94       100
   macro avg       0.94      0.94      0.94       100
weighted avg       0.94      0.94      0.94       100



In [64]:
ix = 0
for twet, asli, pred in zip(twet_uji, label_asli,prediksi):
    if pred!= asli:
        print(ix)
        print("Prediksi:",pred)
        print("asli    :",asli)
        print("    ",twet)
        print("="*100)
    ix+=1

1
Prediksi: positive
asli    : negative
     RT @bagas: Anuuu... pemahamannya kurang eh.. #merdekabelajar
9
Prediksi: positive
asli    : negative
     RT @gw_eilo: Malam Pak @nadiem saya izin tidur ya Pak, ngantuk, bosan sama kebijakan. #merdekabelajar
13
Prediksi: negative
asli    : positive
     RT @CNNIDdaily: Tapi masalah nomor zonasi jadi masalah urutan tuduh-menuduh saya kan juga banyak dituduh pak.
39
Prediksi: negative
asli    : positive
     RT @trenzsolo: "Pak nadiem ini tidak percaya pada guru kita !!" Kick telak atas pernyataan 
44
Prediksi: negative
asli    : positive
     RT @bseptian15: @TeddyGusnaidi Sekarang berani tidak pak @nadiem menolak skripsi dan tesis masuk 
55
Prediksi: negative
asli    : positive
     RT @BagusStefanus: Ketika @nadiem tukang tipu dan korup tapi tetap dijadikan menteri
